In [1]:
import torch
from model import Transformer
from data import vocab_dic, loader
from mask import mask_pad, mask_tril
from tqdm import tqdm

D:\MyAnaconda\envs\a3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading cached processed dataset at wmt_zh-en.hf\cache-1a4ef48df1034150.arrow


In [2]:
print(len(vocab_dic['encode']), len(vocab_dic['decode']))
vocab_dic['encode']['城'], vocab_dic['encode']['市']

12405 12405


(2993, 9259)

In [3]:
vocab_dic['decode'][vocab_dic['encode']['城']], vocab_dic['decode'][vocab_dic['encode']['市']]

('城', '市')

In [4]:
def toStr(data):
  data = data.tolist()
  str = ''
  for idx in data:
    str += vocab_dic['decode'][idx] + ' '
  
  return str.replace('<PAD>', '').replace('<END>', '').replace('<START>', '')

In [5]:
lr = 3e-4
words_size = len(vocab_dic['encode'])
embed_dims = 128
drop_rate = 0.3
heads = 4
sens_len = 80
epochs = 20

In [6]:
transformer = Transformer(words_size=words_size, sens_len=sens_len, heads=heads, embed_dims=embed_dims,
                          drop_rate=drop_rate)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=lr)
transformer.train()
for epoch in range(epochs):
    pbar = tqdm(enumerate(loader), total=len(loader))
    for i, (x, y) in pbar:
        pred = transformer(x, y[:, :-1])
        pred = pred.reshape(-1, words_size)
        y=y[:, 1:].reshape(-1)
        selected = y != vocab_dic['encode']['<PAD>']
        pred = pred[selected]
        y = y[selected]
        
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = pred.argmax(dim=1)
        if epoch == epochs - 1:
            print('y:', toStr(y[:20]))
            print('pred:', toStr(pred[:20]))
        pbar.set_description(f"epoch {epoch + 1} iter {i}: train loss {loss.item():.5f}. lr {lr:e} acc {((pred == y).sum().item())/len(pred)}")

epoch 1 iter 98: train loss 7.41059. lr 3.000000e-04 acc 0.12607449856733524: 100%|████| 99/99 [00:42<00:00,  2.32it/s]
epoch 2 iter 98: train loss 6.93644. lr 3.000000e-04 acc 0.08042895442359249: 100%|████| 99/99 [00:41<00:00,  2.38it/s]
epoch 3 iter 98: train loss 6.51763. lr 3.000000e-04 acc 0.12039312039312039: 100%|████| 99/99 [00:40<00:00,  2.43it/s]
epoch 4 iter 98: train loss 6.62802. lr 3.000000e-04 acc 0.11166253101736973: 100%|████| 99/99 [00:39<00:00,  2.53it/s]
epoch 5 iter 98: train loss 5.94075. lr 3.000000e-04 acc 0.15639810426540285: 100%|████| 99/99 [00:39<00:00,  2.48it/s]
epoch 6 iter 98: train loss 5.52106. lr 3.000000e-04 acc 0.20385674931129477: 100%|████| 99/99 [00:39<00:00,  2.50it/s]
epoch 7 iter 98: train loss 4.79947. lr 3.000000e-04 acc 0.2268041237113402: 100%|█████| 99/99 [00:39<00:00,  2.48it/s]
epoch 8 iter 98: train loss 4.89603. lr 3.000000e-04 acc 0.18925831202046037: 100%|████| 99/99 [00:40<00:00,  2.42it/s]
epoch 9 iter 98: train loss 4.76264. lr 

y: Hogan isn't the first public official to be criticized for defanging Facebook and other social media.  According to our 
pred: Hogan isn't Facebook first public of Facebook be criticized and defanging Facebook and technology with media.  At to the 


epoch 20 iter 1: train loss 1.02034. lr 3.000000e-04 acc 0.7583892617449665:   2%|      | 2/99 [00:00<00:38,  2.53it/s]

y: Emphasis on promotion  Advance the formation of a common destiny for humanity  He stressed that the video and 
pred: Emphasis  promotion  Advance the formation of the common destiny for humanity  In stressed that the video and 


epoch 20 iter 2: train loss 1.30302. lr 3.000000e-04 acc 0.6878048780487804:   3%|▏     | 3/99 [00:01<00:38,  2.47it/s]

...

y: Inject new strong impetus  Wang Fengyuan spoke at the launch of his new book, saying that before coming to 
pred: Inject  strong impetus  She Fengyuan spoke that the experience of the appointment book, said that the the to 


epoch 20 iter 97: train loss 1.60847. lr 3.000000e-04 acc 0.6107226107226107:  99%|███▉| 98/99 [00:39<00:00,  2.51it/s]

y: Yang Yanyin, Director of the Subcommittee of Hong Kong, Macao and Taiwan Compatriots and overseas Chinese individuals, delivered a congratulatory 
pred: Yang Yanyin, of of the Subcommittee of the Kong, Macao is overseas Compatriots of overseas the individuals, delivered of congratulatory 


epoch 20 iter 98: train loss 1.44678. lr 3.000000e-04 acc 0.6649350649350649: 100%|████| 99/99 [00:39<00:00,  2.49it/s]

y: Her captain's speech was incredible," says Black.  We must strive to build a clean and beautiful world with green 
pred: Her captain's speech says incredible," says Black.  We must strive to build a clean and work world in green 


In [7]:
torch.save(transformer.state_dict(), 'transformer.pt')

In [8]:
transformer.load_state_dict(torch.load('transformer.pt'))

<All keys matched successfully>

In [9]:
def predict(x):
    '''
    :param x: [1, n]
    :return: predicted sentence
    '''
    transformer.eval()
    n = x.shape[1]
    mask_pad_x = mask_pad(x)
    pred = [vocab_dic['encode']['<START>']] + [vocab_dic['encode']['<PAD>']] * (n - 1)
    pred = torch.LongTensor(pred).unsqueeze(0)
    x = transformer.embed_x(x)
    x = transformer.encoder(x, mask_pad_x)

    for i in range(n - 1):
        y = pred
        mask_tril_y = mask_tril(y)
        y = transformer.embed_y(y)
        y = transformer.decoder(x, y, mask_pad_x, mask_tril_y)
        out = transformer.fc_out(y)
        out = out[:, i, :].argmax(dim=1).detach()
        pred[:, i + 1] = out
        
#     x = ['<START>'] + list('参加会议') + ['<END>'] + ['<PAD>'] * 74
#     y = ['<START>'] + 'attended the meeting this you'.split(' ') + ['<END>'] + ['<PAD>'] * 73
#     x = [vocab_dic['encode'][w] for w in x]
#     y = [vocab_dic['encode'][w] for w in y]
#     x = torch.LongTensor(x).unsqueeze(0)
#     y = torch.LongTensor(y).unsqueeze(0)
    
#     pred = transformer(x, y).reshape(-1, words_size)
#     y=y.reshape(-1)
#     selected = y != vocab_dic['encode']['<PAD>']
#     pred = pred[selected]
#     y = y[selected]
#     pred = pred.argmax(dim=1)
    return pred
    

In [10]:
for i, (x, y) in enumerate(loader):
    print('x:', toStr(x[0]))
    print('y:', toStr(y[0]))
    print('pred:', toStr(predict(x[0].unsqueeze(0))[0]))
    if i == 5:
        break

x:  胡 塞 武 装 则 声 称 已 经 控 制 了 原 属 于 萨 利 赫 的 今 日 电 视 台 。                                                       
y:  The Houthis armed forces claimed to have taken control of today's television station, which belonged to Saleh.                                                                
pred:  At Riodoce, the Houthis armed forces claimed to control of today's television station, which belonged to Saleh.  government  was taken                                                          
x:  《 十 年 后 》 中 ， 成 年 人 实 实 在 在 的 扮 演 成 年 人 ， 情 绪 便 没 有 太 根 深 蒂 固 ， 模 仿 也 更 加 散 漫 ， 其 中 包 括 对   9 0   年 代 跟 踪 电 影 的 即 兴 创 作 ， 如 《 推 动 摇 篮 的 手 》 。         
y:  "Ten Years Later," with its adults now playing adults, is less emotionally rooted and its parody more scattershot, including a riff on '90s stalker flicks like "The Hand That Rocks the Cradle."                                                 
pred:  In late fact, Paniai rules related to Ferguson by third-party the top of a riff a riff a riff on '90s s

In [11]:
def translate(sent):
    sent = list(sent)
    x = ['<START>'] + sent + ['<END>'] + ['<PAD>'] * 80
    x = x[:80]
    x = [vocab_dic['encode'][w] for w in x]
    x = torch.LongTensor(x).unsqueeze(0)
    return toStr(predict(x)[0])

In [17]:
sent = '中国人不骗中国人。'
translate(sent)

' China China China to sign China.                                                                          '

In [18]:
sent = '你必须以记者的身份承担落到你肩上的任务'
translate(sent)

' "You need to the weird to meet meet meet meet meet meet meet meet meet  to meet                                                              '